# Messages and Requests

## Install

In [ ]:
SYFT_VERSION = ">=0.8.2.b0,<0.9"
package_string = f'"syft{SYFT_VERSION}"'
# %pip install {package_string} -q

In [ ]:
import syft as sy
sy.requires(SYFT_VERSION)

In [ ]:
node = sy.orchestra.launch(name="messages-requests-example-domain-1-do",port=7021, reset=True)

## Setup

For the purpose of this tutorial we are creating a very simple dataset, which is created and owner by the root client

In [ ]:
admin_client = node.login(email="info@openmined.org", password="changethis")

In [ ]:
import numpy as np
dataset = sy.Dataset(
    name="my dataset",
    asset_list=[
        sy.Asset(
        name="my asset",
        data=np.array([1,2,3]),
        mock=np.array([1,1,1])
    )]
)
admin_client.upload_dataset(dataset)

In [ ]:
admin_client.register(name="Jane Doe", email="jane@caltech.edu",
                            password="abc123",password_verify="abc123", institution="Caltech", website="https://www.caltech.edu/")

In [ ]:
guest_client = node.client.login(email="jane@caltech.edu", password="abc123")

In [ ]:
asset = guest_client.datasets[0].assets[0]

In [ ]:
@sy.syft_function_single_use()
def func():
    import random
    data = list(range(100))
    return sum(data) / 100 + random.random()

@sy.syft_function_single_use(data=asset)
def mean(data):
    import random
    return sum(data)/len(data) + + random.random()

@sy.syft_function_single_use(data=asset)
def reveal_data(data):
    return data

In [ ]:
# Create a project
new_project = sy.Project(
    name="My Cool Project",
    description="""Hi, I want to calculate the mean of your private data,\
                    pretty please!""",
    members=[guest_client],
)
new_project

In [ ]:
new_project.create_code_request(func, guest_client)
new_project.create_code_request(mean, guest_client)
new_project.create_code_request(reveal_data, guest_client)

In [ ]:
project = new_project.start()
project

## Messaging

### Check New Messages

In [ ]:
admin_client.notifications

### Send a Message

### Mark as Read or Unread

In [ ]:
message = admin_client.notifications[0]
message

In [ ]:
message.mark_read()

In [ ]:
admin_client.notifications

In [ ]:
message.mark_unread()

In [ ]:
admin_client.notifications

## Requests

### Approving

In [ ]:
admin_client.requests

In [ ]:
func_request = admin_client.requests[-3]
func_request

In [ ]:
func_request.approve()

In [ ]:
admin_client.requests

### Substituting

In [ ]:
mean_request = admin_client.requests[-2]
mean_request

In [ ]:
admin_asset = admin_client.datasets[0].assets[0]

In [ ]:
result = mean_request.code.unsafe_function(data=admin_asset)
result

In [ ]:
mean_request.accept_by_depositing_result(result)
mean_request

In [ ]:
admin_client.projects[0].requests

### Rejecting

In [ ]:
dangerous_request = admin_client.projects[0].requests[-1]
dangerous_request

In [ ]:
dangerous_request.code

In [ ]:
dangerous_request.deny(reason="Dangerous request, you just want to reveal the data!")

In [ ]:
admin_client.projects[0]